In [ ]:
# ============================================================
# Exploratory Raster, NDWI, and Feature Inspection
# ============================================================

from __future__ import annotations
from pathlib import Path

import numpy as np
import geopandas as gpd
import rasterio
from rasterio.warp import transform_bounds
from PIL import Image

import folium
from folium.raster_layers import ImageOverlay

from swmaps.config import data_path
from swmaps.core.indices import compute_ndwi
from swmaps.pipeline.landsat_salinity import estimate_salinity_from_mosaic

In [ ]:
# AOI
GEOJSON_PATH = data_path("config", "choptank_river_region.geojson")

# Multi-band Landsat reflectance mosaic
MOSAIC_PATH = data_path(
    "data",
    "choptank_downloads",
    "landsat-5",
    "landsat-5_LT05_014033_19841124_multiband.tif",
)

OUT_DIR = data_path("notebooks", "explore_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

GEOJSON_PATH, MOSAIC_PATH


In [ ]:
gdf = gpd.read_file(GEOJSON_PATH).to_crs("EPSG:4326")
bbox = gdf.total_bounds

center_lat = float((bbox[1] + bbox[3]) / 2)
center_lon = float((bbox[0] + bbox[2]) / 2)

(center_lat, center_lon)

In [ ]:
def bounds_wgs84(tif: Path) -> list[list[float]]:
    with rasterio.open(tif) as src:
        west, south, east, north = transform_bounds(
            src.crs, "EPSG:4326", *src.bounds, densify_pts=21
        )
    return [[south, west], [north, east]]


def make_map():
    return folium.Map(
        location=[center_lat, center_lon],
        zoom_start=8,
        tiles="CartoDB Positron",
        width="70%",
        height="450px",
    )


def save_png(arr: np.ndarray, path: Path, binary=False):
    arr = np.nan_to_num(arr, nan=0.0).astype(np.float32)
    path.parent.mkdir(parents=True, exist_ok=True)

    if binary:
        arr = np.clip(arr, 0, 1)
        rgb = (np.stack([arr] * 3, axis=-1) * 255).astype(np.uint8)
    else:
        mask = arr != 0
        if np.any(mask):
            lo, hi = np.percentile(arr[mask], (2, 98))
        else:
            lo, hi = 0, 1
        norm = np.clip((arr - lo) / (hi - lo + 1e-6), 0, 1)
        rgb = (np.stack([norm] * 3, axis=-1) * 255).astype(np.uint8)

    Image.fromarray(rgb).save(path)


In [ ]:
with rasterio.open(MOSAIC_PATH) as src:
    print("Bands:", src.count)
    print("CRS:", src.crs)
    print("Dtypes:", src.dtypes)
    print("Descriptions:", src.descriptions)


In [ ]:
bounds = bounds_wgs84(MOSAIC_PATH)
m = make_map()

with rasterio.open(MOSAIC_PATH) as src:
    for i in range(1, src.count + 1):
        arr = src.read(i)
        png = OUT_DIR / f"{MOSAIC_PATH.stem}_band{i}.png"
        save_png(arr, png)
        ImageOverlay(str(png), bounds=bounds, name=f"Band {i}", opacity=0.55).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
m

In [ ]:
NDWI_TIF = OUT_DIR / f"{MOSAIC_PATH.stem}_ndwi.tif"

ndwi = compute_ndwi(
    path=MOSAIC_PATH,
    mission="landsat-5",
    out_path=NDWI_TIF,
    display=False,
)

NDWI_TIF

In [ ]:
bounds_ndwi = bounds_wgs84(NDWI_TIF)
m_ndwi = make_map()

with rasterio.open(NDWI_TIF) as src:
    ndwi_arr = src.read(1)

png = OUT_DIR / f"{NDWI_TIF.stem}.png"
save_png(ndwi_arr, png)

ImageOverlay(png, bounds=bounds_ndwi, name="NDWI", opacity=0.6).add_to(m_ndwi)
folium.LayerControl().add_to(m_ndwi)
m_ndwi

In [ ]:
salinity = estimate_salinity_from_mosaic(
    mosaic_path=MOSAIC_PATH,
    water_threshold=0.2,
)

salinity

In [ ]:
m_sal = make_map()

for name, path in salinity.items():
    path = Path(path)
    bounds_out = bounds_wgs84(path)

    with rasterio.open(path) as src:
        arr = src.read(1)

    png = OUT_DIR / f"{path.stem}.png"
    save_png(arr, png, binary="water_mask" in name)
    ImageOverlay(png, bounds=bounds_out, name=name, opacity=0.6).add_to(m_sal)

folium.LayerControl(collapsed=False).add_to(m_sal)
m_sal